In [14]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder='passthrough')

sgd_pipe = Pipeline([
    ('onehot', onehot_cols),
    ('sgd', XGBRegressor())
])

sgd_pipe = Pipeline([
    ('onehot', onehot_cols),
    ('standardize', StandardScaler()),
    ('sgd', SGDRegressor())
])

In [3]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=sgd_pipe)
    return thesis


In [4]:
thesis = full_run()

Finished experiment! Elapsed time: 0.09678030014038086, total Elapsed time: 0.09678030014038086, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.06183505058288574, total Elapsed time: 0.15961313247680664, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.04992413520812988, total Elapsed time: 0.20953726768493652, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.04684805870056152, total Elapsed time: 0.2573826313018799, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 0.04388236999511719, total Elapsed time: 0.30126500129699707, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift tim

In [5]:
results = thesis.resultsToDF()

In [13]:
results[["SMAPE", "Drift type", "Drift time", "Drift magnitude", "Variable importance"]].groupby(["Drift type", "Drift time", "Drift magnitude", "Variable importance"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                        mean   
Drift type        Drift time     Drift magnitude Variable importance           
Incremental Drift Fully observed Large           Important             5.414   
                                                 Medium                4.314   
                                                 Unimportant           2.058   
                                 Small           Important             2.570   
                                                 Medium                2.216   
                                                 Unimportant           1.974   
                  Half observed  Large           Important            17.906   
                                                 Medium                7.112   
                                                 Unimportant           9.016   
                                 Small           Important             4.620   
                                                 Medium                2.860   
                                                 Unimportant           3.040   
                  Unobserved     Large           Important            13.394   
                                                 Medium                6.180   
                                                 Unimportant           7.556   
                                 Small           Important             3.716   
                                                 Medium                2.588   
                                                 Unimportant           3.078   
No Drift          No Drift       No Drift        No Drift              1.968   
Sudden Drift      Fully observed Large           Important             5.828   
                                                 Medium                4.536   
                                                 Unimportant           2.114   
                                 Small           Important             2.700   
                                                 Medium                2.260   
                                                 Unimportant           1.966   
                  Half observed  Large           Important             8.446   
                                                 Medium                5.834   
                                                 Unimportant           2.632   
                                 Small           Important             3.268   
                                                 Medium                2.610   
                                                 Unimportant           2.042   
                  Unobserved     Large           Important            20.262   
                                                 Medium                8.158   
                                                 Unimportant           9.418   
                                 Small           Important             5.638   
                                                 Medium                3.056   
                                                 Unimportant           3.670   

                                                                                \
                                                                           std   
Drift type        Drift time     Drift magnitude Variable importance             
Incremental Drift Fully observed Large           Important            5.327530   
                                                 Medium               3.552567   
                                                 Unimportant          0.194731   
                                 Small           Important            0.986408   
                                                 Medium               0.415788   
                                                 Unimportant          0.024083   
                  Half observed  Large           Important            5.053284   
                                                 M

In [8]:
results.sort_values("SMAPE", ascending=False)

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
0,None,Sudden Drift,Small,Important,Fully observed,8.560672,1.80
1,None,Sudden Drift,Small,Important,Half observed,9.641532,1.85
2,None,Sudden Drift,Small,Important,Unobserved,55.054172,5.06
3,None,Sudden Drift,Small,Medium,Fully observed,8.856296,1.90
4,None,Sudden Drift,Small,Medium,Half observed,9.546347,1.92
...,...,...,...,...,...,...,...
180,"[0, 3, 6]",Incremental Drift,Large,Medium,Unobserved,529.075514,11.17
181,"[0, 3, 6]",Incremental Drift,Large,Unimportant,Fully observed,13.569438,2.30
182,"[0, 3, 6]",Incremental Drift,Large,Unimportant,Half observed,35.791965,3.34
183,"[0, 3, 6]",Incremental Drift,Large,Unimportant,Unobserved,29.414202,3.02
